In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import time
import os


In [2]:
# Retrieve API key for alpha vantage
mykey = os.environ.get("SECRETKEY")


## Functions to Get Stock Data from Alpha Vantage API and add to Google Sheet

In [3]:
# Function to get the adjusted close share price for the last 100 trading days
# sym is the trading symbol for the stock or index of your choice and the key is your personal 
# alpha vantage api key

def adjusted_close100( sym, key ):
    
    # Code from RapidAPI to connect to alpha vantage API 
    
    url = "https://alpha-vantage.p.rapidapi.com/query"
    
    # Specify stock symbol of interest 
    symbol = sym

    querystring = {"outputsize":"compact","datatype":"json","function":"TIME_SERIES_DAILY_ADJUSTED","symbol":symbol}
    
    
    headers = {
        'x-rapidapi-host': "alpha-vantage.p.rapidapi.com",
        'x-rapidapi-key': key
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    # Make sure responses are in a json format
    responses=response.json()
    
    # Json responses to a pd df with the dates as index
    df = pd.DataFrame.from_dict(responses['Time Series (Daily)'], orient= 'index')
    
    # Select just the column of interest, adjusted close
    df = df[['5. adjusted close']]
    
    # Rename the column and return the dataframe
    df = df.rename(columns={'5. adjusted close': sym})
    return df

In [4]:
# Function to create a new google sheet from a dataframe, df is the dataframe you would like to create a GoogleSheet
# from, sheet_name is what you would like the sheet called, for the purposes of this notebook sheets will
# be named with the trading symbol associated with the data on the sheet
def create_gs(df, sheet_name):
    # Specify the name that you would like the sheet to have in Google Sheets
    wks_name= sheet_name

    # Send dataframe to Google Sheet
    d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
    print('Data upload success')
    

In [5]:
# This towardds article has instructions on sending data to google sheets api for plotting in tableau
# https://towardsdatascience.com/from-analysis-to-dashboarding-connecting-notebooks-to-tableau-483fa373f3a4

scope = ['https://spreadsheets.google.com/feeds',
 'https://www.googleapis.com/auth/drive']

# You will need your key from google to be saved as 'google_secret.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(
 'google_secret.json', scopes=scope)

gc = gspread.authorize(credentials)

# Spreadsheet permissions are managed in google sheets and so this is the id of the spreadsheet not a secret key
spreadsheet_key = '1KEI4RLEvK9xKVkrtjJq5MXsfI8knH67lHXgZSepgIao'

In [12]:
# Check to see that the function is working as expected
df = adjusted_close100('SPY', mykey)

# Add a pause to avoid exceeding 5 requests / min quota of the alpha vantage API free tier
time.sleep(13)

# View top of df 
df.head()

,SPY
2020-07-10,317.5900
2020-07-09,314.3800
2020-07-08,316.1800
2020-07-07,313.7800
2020-07-06,317.0500


In [13]:
# A list of the top rated ESG funds of 2019 from Bloomberg
# https://www.bloomberg.com/graphics/2020-ten-funds-with-a-conscience/

# Using ETF 'SPY' to track the S&P500 stock market index

# List of ESG fund symbols to get data for including S&P500 index data from SPY
symbols = ['MGGPX' , 'BIAWX' , 'MIOPX' , 'CSIEX' , 'AVEGX' , 'PNOPX' , 'AMAGX' , 'MFAPX' , 'PGWAX' , 'IMANX']

# Create the df with the S&P share prices for each date in it
#df = adjusted_close100('SPY', mykey)
# Add a pause to avoid exceeding 5 requests / min quota of the alpha vantage API free tier
#time.sleep(12)

# Create a new column in the df for the adjusted close for each fund
for i in symbols:
    # Get the last 100 days of adjusted close data for the fund
    sym_df = adjusted_close100(i, mykey)
    
    # Join as a new col to the df
    df = df.join(sym_df, how='outer')
    
    # Add a pause to avoid exceeding 5 requests / min quota of the alpha vantage API free tier
    time.sleep(13)


In [14]:
# Make a copy of the df just in case changes need to be made to calculations, 
# the whole thing won't need to be pulled from the api again
df2 = df
df.head()

,SPY,MGGPX,BIAWX,MIOPX,CSIEX,AVEGX,PNOPX,AMAGX,MFAPX,PGWAX,IMANX
2020-02-18,NaN,30.0300,30.1700,26.1000,59.0700,40.3900,NaN,NaN,20.2700,NaN,NaN
2020-02-19,334.7922,30.3400,30.3200,26.4800,59.2500,40.7000,100.6500,46.8700,20.4800,56.9000,14.1100
2020-02-20,333.4168,30.1900,30.0900,26.3100,58.9600,40.4800,100.1700,46.7100,20.3400,56.4600,14.0100
2020-02-21,329.9832,29.8300,29.7100,26.0900,58.4600,40.1100,98.8100,46.3000,20.1900,55.4400,13.8300
2020-02-24,319.0392,28.6900,28.6800,25.1600,56.8000,38.9300,95.5100,44.8000,19.4800,53.5700,13.3400


In [35]:
df = df2

In [36]:
# Make sure dtype is float
df = df.astype(float)

# Calculate daily percent change for each fund
df = df.pct_change()

# Calculate the cumulative returns for each fund
df = (df + 1).cumprod()

# We want the returns as a percent change over the last 100 days
df = -1 * (1 - df)

df.head()

,SPY,MGGPX,BIAWX,MIOPX,CSIEX,AVEGX,PNOPX,AMAGX,MFAPX,PGWAX,IMANX
2020-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-19,NaN,0.010323,0.004972,0.014559,0.003047,0.007675,NaN,NaN,0.010360,NaN,NaN
2020-02-20,-0.004108,0.005328,-0.002652,0.008046,-0.001862,0.002228,-0.004769,-0.003414,0.003453,-0.007733,-0.007087
2020-02-21,-0.014364,-0.006660,-0.015247,-0.000383,-0.010327,-0.006932,-0.018281,-0.012161,-0.003947,-0.025659,-0.019844
2020-02-24,-0.047053,-0.044622,-0.049387,-0.036015,-0.038429,-0.036148,-0.051068,-0.044165,-0.038974,-0.058524,-0.054571


In [37]:
# Load the df into a GoogleSheet
create_gs(df, 'ESG_last100')

Data upload success
